In [36]:
import requests
from bs4 import BeautifulSoup as soup
import pandas as pd
import uuid
import difflib

In [37]:
player_index = pd.read_csv('../updated_datasets/player_index.csv')
player_index = player_index[player_index.columns[1:]]
player_index

,Player,UUID
0,Kevin Garnett,4035d838-5ed7-4507-86ad-0488a9b9c358
1,Shaquille O'Neal,74b99a51-bb65-436f-8bcc-7022af2a490a
2,Alonzo Mourning,8051eb29-ecd7-46b5-b2fa-415777e11e83
3,Juwan Howard,5bdd7fb5-050d-4b0c-befb-5fbf7109397c
4,Hakeem Olajuwon,be9d9c53-d445-4b8c-acc9-ad1c30e8598b
...,...,...
2224,Jordan Schakel,49852ad9-c5e3-4124-bd75-2ad27b5a859f
2225,Jordan Goodwin,70d9ad35-fb19-497a-a358-d77a2a2e89ba
2226,Craig Sword,c29acdbe-9077-44c0-a8b3-f997ddff72a2
2227,Jaime Echenique,727da240-fbce-4643-ba94-f74509e2e746


In [38]:
null_players = pd.read_csv('../updated_datasets/null_players.csv')
null_players = null_players[null_players.columns[1:]]
null_players.head()

,New_name,Original_name
0,Darrell Armstrong,Darrel Armstrong
1,Predrag Stojakovic,Peja Stojakovic
2,PJ Brown,P.J. Brown
3,Jaren Jackson Jr,Jaren Jackson
4,AJ Guyton,A.J. Guyton


In [3]:
table_list = []
df_list = []
def awards_data(website):
    main = soup(requests.get(website).text, 'html.parser')
    links = [a['href'] for a in main.find_all('a', class_='bi', href=True)]
    links.remove('//www.espn.com/nba/history/awards/_/id/34')
    
    for link in links:
        #grab the header of the website page
        name = str(soup(requests.get(f'http:{link}').text).select('h2')[0]).replace(
            '<h2>', '').replace('</h2>', '').replace('<h2>', '').partition('- ')[-1].split()
        
        #'the' doesn't get iterated through so remove it
        if 'the' in name:
            name.remove('the')
            
        #set variable name to the first two capitalized words of the header
        if len(name) > 1:
            for word in name:
                if word[0] != word[0].upper():
                    name.remove(word)
            var_name = (name[0] + '_' + name[1]).lower().replace('-', '_')
        else:
            var_name = ''.join(name).lower()
        
        table_list.append(var_name)
        
        #grab the datset
        globals()[var_name] = pd.read_html(requests.get(f'http:{link}').text)[0]
        
        #clean the data
        globals()[var_name].columns = globals()[var_name].iloc[1].str.capitalize()
        globals()[var_name] = globals()[var_name].tail(-2)

        #some tables have blank year columns so filter for that
        globals()[var_name].fillna(method='ffill', inplace=True)

        #more cleaning
        globals()[var_name] = globals()[var_name][globals()[var_name]['Year'].astype('int') >= 2000]
        globals()[var_name] = globals()[var_name][globals()[var_name].columns[:4]]

        #gather uuids
        globals()[var_name] = pd.merge(left=globals()[var_name], right=player_index, how='left', on='Player')
        
        #locate mispelled names
        null_players = []
        for row in globals()[var_name][globals()[var_name]['UUID'].isnull()]['Player']:
            null_players.append(row)
        null_players = pd.DataFrame(null_players, columns=['Player']).drop_duplicates()
        
        #locate correct names from index
        nearest_name = []
        original_name = []
        for name in null_players['Player']:
            close_matches = difflib.get_close_matches(name, player_index['Player'], 1)
            for x in close_matches:
                    nearest_name.append(x)
                    original_name.append(name)
        nearest_name = pd.DataFrame(nearest_name, columns=['New_name'])
        nearest_name['Original_name'] = original_name
        
        #fix mispelled names 
        for bad_name in globals()[var_name]['Player']:
            if bad_name in [x for x in nearest_name['Original_name']]:
                globals()[var_name].loc[globals()[var_name][globals()[var_name]['Player'] == bad_name].index, 'Player'] = [
                    x for x in nearest_name[nearest_name['Original_name'] == bad_name]['New_name']][0]
        
        #merge w/ index table to add unique ids and save csvs
        globals()[var_name] = pd.merge(left=globals()[var_name], right=player_index, how='left', on='Player')
        globals()[var_name].drop(columns=['UUID_x'], inplace=True)
        globals()[var_name].rename(columns={'UUID_y':'UUID'}, inplace=True)
        globals()[var_name].to_csv(f'../updated_datasets/awards_data/{var_name}.csv')
        
        #add dataframe to my output list of dataframes
        df_list.append(globals()[var_name])

    return table_list

In [4]:
awards_data('http://www.espn.com/nba/history/awards')

['mvp',
 'defensive_player',
 'rookie_year',
 'sixth_man',
 'most_improved',
 'finals_mvp',
 'all_star_mvp',
 'all_nba_1st',
 'all_nba_2nd',
 'all_nba_3rd',
 'all_rookie_1st',
 'all_rookie_2nd',
 'all_defensive_1st',
 'all_defensive_2nd']

In [5]:
sixth_man.head()

,Year,Player,Pos,Team,UUID
0,2022,Tyler Herro,PG,Miami Heat,b1359ec1-999b-418f-84e8-791659e3d59c
1,2021,Jordan Clarkson,PG,Utah Jazz,d7aa0308-767f-4f9a-905e-8800e7c41ef7
2,2020,Montrezl Harrell,F,LA Clippers,64379148-f232-45b7-abe0-c3f3b5c6138a
3,2019,Louis Williams,G,LA Clippers,d861edd2-969c-4d79-ae0d-9c6d1ffa11d2
4,2018,Louis Williams,G,LA Clippers,d861edd2-969c-4d79-ae0d-9c6d1ffa11d2


In [6]:
mvp.head()

,Year,Player,Pos,Team,UUID
0,2022,Nikola Jokic,C,Denver Nuggets,4d321b07-ade4-497c-be07-e8335e03b288
1,2021,Nikola Jokic,C,Denver Nuggets,4d321b07-ade4-497c-be07-e8335e03b288
2,2020,Giannis Antetokounmpo,F,Milwaukee Bucks,f5e976f0-2383-4f34-869e-912b0daebc1d
3,2019,Giannis Antetokounmpo,F,Milwaukee Bucks,f5e976f0-2383-4f34-869e-912b0daebc1d
4,2018,James Harden,G,Houston Rockets,ef0995d8-4803-48be-b6b8-d6cdc7e8c433


In [7]:
all_rookie_1st.loc[all_rookie_1st[all_rookie_1st['Player'] == 'Nene Hilario'].index, 'Player'] = 'Nenê'

In [8]:
all_rookie_1st.loc[all_rookie_1st[all_rookie_1st['Player'] == 'Nenê'].index, 'UUID'] = '71b3fc68-5b92-4644-af01-caa989d432ad'

In [9]:
#readd_uuids(all_rookie_1st)

In [10]:
all_rookie_1st[all_rookie_1st['Player'] == 'Nenê']

,Year,Player,Pos,Team,UUID
100,2003,Nenê,C,Denver Nuggets,71b3fc68-5b92-4644-af01-caa989d432ad


In [11]:
def define_nulls(tables):
    null_players = []
    for table in tables:
        for row in table[table['UUID'].isnull()]['Player']:
            null_players.append(row)
    null_players = pd.DataFrame(null_players, columns=['Player']).drop_duplicates()
    return null_players

In [12]:
define_nulls(df_list)

,Player


In [13]:
def search_index(table):
    return sum([True if len(cell) > 0 else False for cell in [player_index[player_index['Player'] == row] for row in table['Player']]])

In [2]:
def player_lookup(players, table):
    player_lookup = pd.DataFrame(columns = ['Player', 'UUID'])
    player_name = []
    player_uuid = []
    for player in players:
            for name in table['Player']:
                if player.lower() in name.lower():
                    player_name.append(name)
                    for uuid in table[table['Player'] == name]['UUID']:
                        player_uuid.append(uuid)
    player_lookup['Player'] = player_name
    player_lookup['UUID'] = player_uuid
    return player_lookup

In [21]:
player_lookup(['curry'], player_index)

,Player,UUID
0,Michael Curry,10edaf26-b605-4241-9ff9-860ea23101d6
1,Dell Curry,0ad833c1-5cc6-4eb9-80d6-dd3b9f3b3a19
2,Eddy Curry,5c066c64-425d-4ae2-ba7b-38715634f032
3,JamesOn Curry,af58af9b-67fd-4817-8684-7c62b8f6cc4f
4,Stephen Curry,2f1008df-4d89-43d1-b8be-b9f05520f109
5,Seth Curry,cccb6627-2815-4fe9-a8f0-848a8612a022


In [ ]:
nearest_name(null_players['Player'])

In [ ]:
table_list

In [ ]:
def patch_nulls(tables):
    names = nearest_name(define_nulls(tables)['Player'])    
    names['Check'] = ''
    for table in tables:
        for bad_name in table['Player']:
            if bad_name in [x for x in names['Original_name']]:
                table.loc[table[table['Player'] == bad_name].index, 'Player'] = [
                    x for x in names[names['Original_name'] == bad_name]['New_name']][0]
                names['Check'] = True
        table = pd.merge(left=table, right=player_index, how='left', on='Player')
    return names

In [ ]:
#patch_nulls(df_list)

In [ ]:
player_lookup(['Nen'], player_index)

In [ ]:
all_rookie_1st[all_rookie_1st['Player'] == 'Nenê']

In [ ]:
player_index[player_index['Player'] == 'Nenê']

In [42]:
def grab_links(website):
    links = []
    main = soup(requests.get('http://www.espn.com/nba/history/awards').text, 'html.parser')
    for a in main.find_all('a', class_='bi', href=True):
        links.append(a['href'])
    links.remove('//www.espn.com/nba/history/awards/_/id/34')
    return links

In [55]:
def define_table_names(link):
    table_list = []
    #grab the header of the website page
    name = str(soup(requests.get(f'http:{link}').text).select('h2')[0]).replace(
        '<h2>', '').replace('</h2>', '').replace('<h2>', '').partition('- ')[-1].split()

    #'the' doesn't get iterated through so remove it
    if 'the' in name:
        name.remove('the')

    #set variable name to the first two capitalized words of the header
    if len(name) > 1:
        for word in name:
            if word[0] != word[0].upper():
                name.remove(word)
        var_name = (name[0] + '_' + name[1]).lower().replace('-', '_')
    else:
        var_name = ''.join(name).lower()

    table_list.append(var_name)
    return table_list

In [44]:
def gather_data(link):
    #grab the datset
    globals()[var_name] = pd.read_html(requests.get(f'http:{link}').text)[0]

In [45]:
def clean_data(table):
    table.columns = table.iloc[1].str.capitalize()
    table = table.tail(-2)    
    table.fillna(method='ffill', inplace=True) #some tables have blank year columns so filter for that
    table = table[table['Year'].astype('int') >= 2000]
    table = table[table.columns[:4]]

In [46]:
def merge_uuids(table):
    table = pd.merge(left=table, right=player_index, how='left', on='Player')

In [47]:
def mispelled_names(table):
    for row in table[table['UUID'].isnull()]['Player']:
        null_players.append(row)
    null_players = pd.DataFrame(null_players, columns=['Player']).drop_duplicates()

In [48]:
def nearest_name(players):
    nearest_name = []
    original_name = []
    for name in players['Player']:
        close_matches = difflib.get_close_matches(name, player_index['Player'], 1)
        for x in close_matches:
                nearest_name.append(x)
                original_name.append(name)
    nearest_name = pd.DataFrame(nearest_name, columns=['New_name'])
    nearest_name['Original_name'] = original_name

In [49]:
def correct_names(table):
    for bad_name in table['Player']:
        if bad_name in [x for x in null_players['Original_name']]:
            table.loc[table[table['Player'] == bad_name].index, 'Player'] = [
                x for x in null_players[null_players['Original_name'] == bad_name]['New_name']][0]

In [50]:
def readd_uuids(table):
    table = pd.merge(left=table, right=player_index, how='left', on='Player')
    table.drop(columns=['UUID_x'], inplace=True)
    table.rename(columns={'UUID_y':'UUID'}, inplace=True)
    return table

In [59]:
table_list = []
df_list = []
def awards_data(website):
    main = soup(requests.get(website).text, 'html.parser')
    links = [a['href'] for a in main.find_all('a', class_='bi', href=True)]
    links.remove('//www.espn.com/nba/history/awards/_/id/34')
    
    for link in links:
        #grab the header of the website page
        name = str(soup(requests.get(f'http:{link}').text).select('h2')[0]).replace(
            '<h2>', '').replace('</h2>', '').replace('<h2>', '').partition('- ')[-1].split()
        
        #'the' doesn't get iterated through so remove it
        if 'the' in name:
            name.remove('the')
            
        #set variable name to the first two capitalized words of the header
        if len(name) > 1:
            for word in name:
                if word[0] != word[0].upper():
                    name.remove(word)
            var_name = (name[0] + '_' + name[1]).lower().replace('-', '_')
        else:
            var_name = ''.join(name).lower()
        
        table_list.append(var_name)
        
        #grab the datset
        globals()[var_name] = pd.read_html(requests.get(f'http:{link}').text)[0]
        
        #clean the data
        globals()[var_name].columns = globals()[var_name].iloc[1].str.capitalize()
        globals()[var_name] = globals()[var_name].tail(-2)

        #some tables have blank year columns so filter for that
        globals()[var_name].fillna(method='ffill', inplace=True)

        #more cleaning
        globals()[var_name] = globals()[var_name][globals()[var_name]['Year'].astype('int') >= 2000]
        globals()[var_name] = globals()[var_name][globals()[var_name].columns[:4]]

        #gather uuids
        globals()[var_name] = pd.merge(left=globals()[var_name], right=player_index, how='left', on='Player')
        
        #fix mispelled names 
        for bad_name in globals()[var_name]['Player']:
            if bad_name in [x for x in null_players['Original_name']]:
                globals()[var_name].loc[globals()[var_name][globals()[var_name]['Player'] == bad_name].index, 'Player'] = [
                    x for x in null_players[null_players['Original_name'] == bad_name]['New_name']][0]
        
        #merge w/ index table to add unique ids and save csvs
        globals()[var_name] = pd.merge(left=globals()[var_name], right=player_index, how='left', on='Player')
        globals()[var_name].dropna()
        globals()[var_name].drop(columns=['UUID_x'], inplace=True)
        globals()[var_name].rename(columns={'UUID_y':'UUID'}, inplace=True)
        globals()[var_name].to_csv(f'../updated_datasets/awards_data/{var_name}.csv')
        
        #add dataframe to my output list of dataframes
        df_list.append(globals()[var_name])

    return table_list

In [60]:
awards_data('http://www.espn.com/nba/history/awards')

['mvp',
 'defensive_player',
 'rookie_year',
 'sixth_man',
 'most_improved',
 'finals_mvp',
 'all_star_mvp',
 'all_nba_1st',
 'all_nba_2nd',
 'all_nba_3rd',
 'all_rookie_1st',
 'all_rookie_2nd',
 'all_defensive_1st',
 'all_defensive_2nd']